In [17]:
import tempfile
import subprocess
import re

def format_overnight(lf):
    replacements = [
        ('(', ' ( '), # make sure ( and ) must have blank space around
        (')', ' ) '),
        ('! ', '!'),
        ('SW', 'edu.stanford.nlp.sempre.overnight.SimpleWorld'),
    ]
    for a, b in replacements:
        lf = lf.replace(a, b)
    # remove redundant blank spaces
    lf = re.sub(' +', ' ', lf)
    return lf.strip()

def normalize(lf_list):
    return [format_overnight(lf) for lf in lf_list]

def obtain_denotations(lf_list, domain):
    tf = tempfile.NamedTemporaryFile('w+t', encoding='utf8', suffix='.examples')
    for line in lf_list:
        tf.write(line + '\n')
    tf.flush()
    msg = subprocess.check_output(['./data/overnight/evaluator/overnight', domain, tf.name])
    msg = msg.decode('utf8')
    tf.close()
    denotations = [
        line.split('\t')[1] for line in msg.split('\n')
        if line.startswith('targetValue\t')
    ]
    return denotations

def is_valid(ans):
    return not ('BADJAVA' in ans or 'ERROR' in ans or 'Exception' in ans or ans == 'null')

In [1]:
from transformers import AutoTokenizer
import pickle as pkl

In [2]:
inputs = []
with open("./exp_files/webnlg_reverse/reordered/val.pt", 'rb') as f:
    for _ in range(5):
        inputs.append(pkl.load(f))

In [3]:
from data.webnlg_reverse.config import special_tokens
tokenizer = AutoTokenizer.from_pretrained('t5-base')
tokenizer.add_tokens(special_tokens)

7

In [4]:
from data.webnlg_reverse.evaluate import *
targets = [tokenizer.decode(inputs[2][i], skip_special_tokens=True) for i in range(len(inputs[2]))]

In [5]:
back = [post_process(i) for i in targets]

In [6]:
back

['<H> Aarhus <R> leader Name <T> Jacob Bundsgaard',
 '<H> Aarhus Airport <R> runway Length <T> 2702.0',
 '<H> Adirondack Regional Airport <R> elevation Above The Sea Level in metres <T> 507',
 '<H> Adirondack Regional Airport <R> location <T> Harrietstown New York',
 '<H> Adolfo Suarez Madrid-Barajas Airport <R> location <T> San Sebastian de los Reyes',
 '<H> Adolfo Suarez Madrid-Barajas Airport <R> runway Name <T> 14L/32R',
 '<H> Adolfo Suarez Madrid-Barajas Airport <R> runway Name <T> 14R/32L',
 '<H> Afonso Pena International Airport <R> operating Organisation <T> Infraero',
 '<H> Agra Airport <R> location <T> Agra',
 '<H> Al-Taqaddum Air Base <R> elevation Above The Sea Level in metres <T> 84.0',
 '<H> Alderney Airport <R> city Served <T> Alderney',
 '<H> Allama Iqbal International Airport <R> runway Name <T> 18R/36L',
 '<H> Alpena County Regional Airport <R> runway Length <T> 1533.0',
 '<H> Alpena County Regional Airport <R> runway Name <T> 1/19',
 '<H> Amsterdam Airport Schiphol <